In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00


In [8]:
import os
import cv2
from ultralytics import RTDETR

# ---------------- CONFIG ----------------
INPUT_DIR = "/content/drive/MyDrive/Project/test_violence_dataset"          # your input folder
OUTPUT_DIR = "/content/drive/MyDrive/Project/blurred4_output"        # where to save processed images
MODEL_NAME = "rtdetr-l.pt"            # or "rtdetr-x.pt" (auto-downloads)
CONF_THRESH = 0.35                    # only blur detections ≥ this confidence
BLUR_MODE = "gaussian"                # "gaussian" or "pixelate"
CLASSES_TO_BLUR = None                # e.g. ["person"] or None to blur all
PIXELATE_FACTOR = 12                  # bigger = chunkier pixels
GAUSS_MIN_K = 31                      # minimum odd kernel size for Gaussian
# ---------------------------------------

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load pretrained RT-DETR (Ultralytics will fetch weights automatically)
model = RTDETR(MODEL_NAME)

# Pre-compute a lookup from class name -> id (for COCO-pretrained models)
names = model.names  # dict: {id: "class_name"}
name_to_id = {v: k for k, v in names.items()}

# If user provided class names, map them to ids (ignore unknowns)
if CLASSES_TO_BLUR:
    class_ids_to_blur = {name_to_id[c] for c in CLASSES_TO_BLUR if c in name_to_id}
else:
    class_ids_to_blur = None  # blur everything

def odd(n: int) -> int:
    # make n odd (Gaussian kernel must be odd)
    return n if n % 2 == 1 else n + 1

def blur_roi(roi, mode="gaussian"):
    if mode == "pixelate":
        h, w = roi.shape[:2]
        # avoid zero sizes
        small_w = max(1, w // PIXELATE_FACTOR)
        small_h = max(1, h // PIXELATE_FACTOR)
        tmp = cv2.resize(roi, (small_w, small_h), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(tmp, (w, h), interpolation=cv2.INTER_NEAREST)
    else:
        # pick kernel based on box size for nicer results
        h, w = roi.shape[:2]
        k = odd(max(GAUSS_MIN_K, min(w, h) // 5))
        return cv2.GaussianBlur(roi, (k, k), 0)

# Collect images
image_files = [f for f in os.listdir(INPUT_DIR)
               if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"))]

for i, fname in enumerate(sorted(image_files), 1):
    in_path = os.path.join(INPUT_DIR, fname)
    img = cv2.imread(in_path)
    if img is None:
        print(f"[skip] Cannot read: {fname}")
        continue

    # Run detection (returns Ultralytics-style results)
    results = model.predict(source=img, conf=CONF_THRESH, verbose=False)

    if not results:
        out_path = os.path.join(OUTPUT_DIR, fname)
        cv2.imwrite(out_path, img)
        print(f"[{i}/{len(image_files)}] {fname}: no detections -> saved")
        continue

    r = results[0]
    if r.boxes is None or len(r.boxes) == 0:
        out_path = os.path.join(OUTPUT_DIR, fname)
        cv2.imwrite(out_path, img)
        print(f"[{i}/{len(image_files)}] {fname}: no boxes -> saved")
        continue

    H, W = img.shape[:2]

    for box in r.boxes:
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])

        # class filter (if provided)
        if class_ids_to_blur is not None and cls_id not in class_ids_to_blur:
            continue

        # coords
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        # clamp to image bounds
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(W, x2), min(H, y2)

        if x2 <= x1 or y2 <= y1:
            continue

        roi = img[y1:y2, x1:x2]
        if roi.size == 0:
            continue

        roi_blurred = blur_roi(roi, BLUR_MODE)
        img[y1:y2, x1:x2] = roi_blurred

    out_path = os.path.join(OUTPUT_DIR, fname)
    cv2.imwrite(out_path, img)
    print(f"[{i}/{len(image_files)}] {fname} -> {out_path}")

print("Done. Blurred images saved to:", OUTPUT_DIR)


[1/68] video10-mp4_segment_3-mp4_1625_jpg.rf.9f1c4ef9d498f0c558db9caa91b82460.jpg -> /content/drive/MyDrive/Project/blurred4_output/video10-mp4_segment_3-mp4_1625_jpg.rf.9f1c4ef9d498f0c558db9caa91b82460.jpg
[2/68] video12-mp4_segment_4-mp4_750_jpg.rf.15c3f261eed32331e5f36c637f1985a6.jpg -> /content/drive/MyDrive/Project/blurred4_output/video12-mp4_segment_4-mp4_750_jpg.rf.15c3f261eed32331e5f36c637f1985a6.jpg
[3/68] video2-mp4_segment_12-mp4_80_jpg.rf.af77e55f1e5ae0eb99b3014fb340e508.jpg -> /content/drive/MyDrive/Project/blurred4_output/video2-mp4_segment_12-mp4_80_jpg.rf.af77e55f1e5ae0eb99b3014fb340e508.jpg
[4/68] video3-mp4_segment_11-mp4_15_jpg.rf.3c0dc2f72c55aca5ca40bc0be7e55d22.jpg -> /content/drive/MyDrive/Project/blurred4_output/video3-mp4_segment_11-mp4_15_jpg.rf.3c0dc2f72c55aca5ca40bc0be7e55d22.jpg
[5/68] video3-mp4_segment_5-mp4_535_jpg.rf.24189247d74bbc5a37214ed904b587b3.jpg -> /content/drive/MyDrive/Project/blurred4_output/video3-mp4_segment_5-mp4_535_jpg.rf.24189247d74bbc5a